In [21]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler, StandardScaler, RobustScaler, IndexToString
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import mlflow
#from mlflow import pyspark
from mlflow.tracking import MlflowClient

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'classifyHW'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [3]:
# load the data
fil = '../../data/beatsdataset.csv'
beats = spark.read.format('csv').options(header=True, inferSchema=True).load(fil).drop('_c0')
print('%d records'%beats.count())
display(beats.limit(10).toPandas())
beats.printSchema()

2300 records


,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,10-MFCCs2m,11-MFCCs3m,12-MFCCs4m,13-MFCCs5m,14-MFCCs6m,15-MFCCs7m,16-MFCCs8m,17-MFCCs9m,18-MFCCs10m,19-MFCCs11m,20-MFCCs12m,21-MFCCs13m,22-ChromaVector1m,23-ChromaVector2m,24-ChromaVector3m,25-ChromaVector4m,26-ChromaVector5m,27-ChromaVector6m,28-ChromaVector7m,29-ChromaVector8m,30-ChromaVector9m,31-ChromaVector10m,32-ChromaVector11m,33-ChromaVector12m,34-ChromaDeviationm,35-ZCRstd,36-Energystd,37-EnergyEntropystd,38-SpectralCentroidstd,39-SpectralSpreadstd,40-SpectralEntropystd,41-SpectralFluxstd,42-SpectralRolloffstd,43-MFCCs1std,44-MFCCs2std,45-MFCCs3std,46-MFCCs4std,47-MFCCs5std,48-MFCCs6std,49-MFCCs7std,50-MFCCs8std,51-MFCCs9std,52-MFCCs10std,53-MFCCs11std,54-MFCCs12std,55-MFCCs13std,56-ChromaVector1std,57-ChromaVector2std,58-ChromaVector3std,59-ChromaVector4std,60-ChromaVector5std,61-ChromaVector6std,62-ChromaVector7std,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,1.594074,0.011276,0.204468,0.042072,0.048552,0.158505,0.118984,-0.147956,-0.186152,-0.026418,-0.007264,-0.017900,0.011581,0.008747,0.041081,0.014497,0.025711,0.012587,0.060170,0.002864,0.004631,0.009576,0.026079,0.004161,0.032185,0.050143,0.047313,0.102995,0.041285,0.017725,0.414831,0.005867,0.133778,0.838302,0.505911,0.356206,0.336074,0.288888,0.278649,0.283437,0.300305,0.287688,0.296692,0.258531,0.238352,0.194701,0.013138,0.011665,0.032049,0.015464,0.020453,0.012943,0.046397,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,1.261364,-0.113015,0.001718,-0.052682,0.204130,0.153013,0.067214,-0.013227,-0.059440,-0.008604,0.114257,0.171009,0.006535,0.002646,0.086485,0.008391,0.016442,0.009006,0.087948,0.002472,0.006549,0.007412,0.015386,0.005978,0.041116,0.043713,0.043721,0.099449,0.039386,0.018946,0.407164,0.003613,0.110334,0.624185,0.476993,0.353151,0.335550,0.283832,0.269621,0.244150,0.246660,0.257190,0.272036,0.269477,0.222393,0.187471,0.006761,0.003152,0.058923,0.009012,0.016106,0.009386,0.071726,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,1.425185,0.186749,0.417114,0.076406,0.190803,-0.016302,0.075038,0.107870,0.216874,0.095604,0.020977,-0.037011,0.007143,0.002960,0.220526,0.005639,0.010151,0.007453,0.043907,0.001240,0.004347,0.007989,0.017622,0.002636,0.066049,0.032920,0.037618,0.117704,0.041509,0.022645,0.340130,0.007697,0.085784,1.028740,0.449133,0.297935,0.266731,0.258299,0.275012,0.218368,0.198390,0.210177,0.212533,0.204458,0.197634,0.164910,0.007836,0.003079,0.093865,0.005692,0.008212,0.005451,0.042900,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,1.463686,0.226548,0.404531,0.117699,0.081861,0.053974,0.164865,0.014919,0.117090,0.027778,-0.063173,-0.052606,0.010724,0.003340,0.125459,0.005728,0.014695,0.006322,0.072154,0.001628,0.003493,0.011463,0.032204,0.004738,0.046159,0.036349,0.061960,0.134908,0.032564,0.020036,0.365068,0.005215,0.086336,0.769981,0.425496,0.245312,0.260132,0.224220,0.207597,0.199472,0.207818,0.189912,0.185509,0.187273,0.177629,0.164740,0.008330,0.003528,0.061426,0.005443,0.012382,0.004985,0.057999,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,1.187854,0.184415,0.363724,0.232119,0.112277,0.107335,0.159296,0.067213,-0.018713,0.091529,0.117344,0.091616,0.009624,0.004031,0.076133,0.008175,0.016267,0.009927

root
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (nullable = true)
 |-- 23-ChromaVector2m:

In [4]:
# specify the response colum & tabulate by it
responseVar = 'class'
display(beats.groupBy(responseVar).count().toPandas())

,class,count
0,PsyTrance,100
1,HardDance,100
2,Breaks,100
3,HardcoreHardTechno,100
4,IndieDanceNuDisco,100
5,Trance,100
6,DeepHouse,100
7,ElectronicaDowntempo,100
8,ReggaeDub,100
9,Minimal,100


In [5]:
# check for missing values
cnt = beats.count()
nullCounts = {colm:beats.select(colm).where(col(colm).isNull()).count() for colm in beats.columns}
nullCounts = {colm:(ncnt, ncnt/cnt) for (colm, ncnt) in nullCounts.items()}

# pretty print
nullCountsDF = pd.DataFrame(nullCounts).T.reset_index(drop=False).sort_values(1, ascending=False)
nullCountsDF.columns = ['Column', 'Freq.', 'Rel. Freq.']
nullCountsDF = nullCountsDF.merge(pd.DataFrame([[colm.name, colm.dataType] for colm in beats.schema], columns=['Column', 'Type']),
                                how='inner', on=['Column'])
display(nullCountsDF)

,Column,Freq.,Rel. Freq.,Type
0,1-ZCRm,0.0,0.0,DoubleType
1,2-Energym,0.0,0.0,DoubleType
2,53-MFCCs11std,0.0,0.0,DoubleType
3,52-MFCCs10std,0.0,0.0,DoubleType
4,51-MFCCs9std,0.0,0.0,DoubleType
5,50-MFCCs8std,0.0,0.0,DoubleType
6,49-MFCCs7std,0.0,0.0,DoubleType
7,48-MFCCs6std,0.0,0.0,DoubleType
8,47-MFCCs5std,0.0,0.0,DoubleType
9,46-MFCCs4std,0.0,0.0,DoubleType


In [6]:
''' prep the data for modeling '''
# set inputs
inpColumns = beats.columns[:-1]

# create the features vector
assr = VectorAssembler(inputCols=inpColumns, outputCol='features_orig')
beatsML = assr.transform(beats)

# scale the features
scalr = RobustScaler(inputCol='features_orig', outputCol='features')
beatsML = scalr.fit(beatsML).transform(beatsML)

# make the response numerical
indxr = StringIndexer(inputCol=responseVar, outputCol='label')
beatsML = indxr.fit(beatsML).transform(beatsML)

# get a distinct list of the original labels
origLabels = beatsML.select('label').distinct().orderBy('label')
rev = IndexToString(inputCol='label', outputCol='label_orig')
origLabels = [c['label_orig'] for c in rev.transform(origLabels).select('label_orig').collect()]

# talk
display(beatsML.limit(10).toPandas())
beatsML.select('features', 'label').show(truncate=True)
beatsML.select('features').take(1)
print('First row features = %s'%beatsML.select('features').take(1)[0])

,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,10-MFCCs2m,11-MFCCs3m,12-MFCCs4m,13-MFCCs5m,14-MFCCs6m,15-MFCCs7m,16-MFCCs8m,17-MFCCs9m,18-MFCCs10m,19-MFCCs11m,20-MFCCs12m,21-MFCCs13m,22-ChromaVector1m,23-ChromaVector2m,24-ChromaVector3m,25-ChromaVector4m,26-ChromaVector5m,27-ChromaVector6m,28-ChromaVector7m,29-ChromaVector8m,30-ChromaVector9m,31-ChromaVector10m,32-ChromaVector11m,33-ChromaVector12m,34-ChromaDeviationm,35-ZCRstd,36-Energystd,37-EnergyEntropystd,38-SpectralCentroidstd,39-SpectralSpreadstd,40-SpectralEntropystd,41-SpectralFluxstd,42-SpectralRolloffstd,43-MFCCs1std,44-MFCCs2std,45-MFCCs3std,46-MFCCs4std,47-MFCCs5std,48-MFCCs6std,49-MFCCs7std,50-MFCCs8std,51-MFCCs9std,52-MFCCs10std,53-MFCCs11std,54-MFCCs12std,55-MFCCs13std,56-ChromaVector1std,57-ChromaVector2std,58-ChromaVector3std,59-ChromaVector4std,60-ChromaVector5std,61-ChromaVector6std,62-ChromaVector7std,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class,features_orig,features,label
0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,1.594074,0.011276,0.204468,0.042072,0.048552,0.158505,0.118984,-0.147956,-0.186152,-0.026418,-0.007264,-0.017900,0.011581,0.008747,0.041081,0.014497,0.025711,0.012587,0.060170,0.002864,0.004631,0.009576,0.026079,0.004161,0.032185,0.050143,0.047313,0.102995,0.041285,0.017725,0.414831,0.005867,0.133778,0.838302,0.505911,0.356206,0.336074,0.288888,0.278649,0.283437,0.300305,0.287688,0.296692,0.258531,0.238352,0.194701,0.013138,0.011665,0.032049,0.015464,0.020453,0.012943,0.046397,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom,"[0.136439587512, 0.0888612604609, 3.2012005559...","[2.621763736423596, 1.70103481603904, 32.56325...",0.0
1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,1.261364,-0.113015,0.001718,-0.052682,0.204130,0.153013,0.067214,-0.013227,-0.059440,-0.008604,0.114257,0.171009,0.006535,0.002646,0.086485,0.008391,0.016442,0.009006,0.087948,0.002472,0.006549,0.007412,0.015386,0.005978,0.041116,0.043713,0.043721,0.099449,0.039386,0.018946,0.407164,0.003613,0.110334,0.624185,0.476993,0.353151,0.335550,0.283832,0.269621,0.244150,0.246660,0.257190,0.272036,0.269477,0.222393,0.187471,0.006761,0.003152,0.058923,0.009012,0.016106,0.009386,0.071726,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom,"[0.117038518483, 0.108389033282, 3.19400106287...","[2.2489612371225074, 2.074846995565889, 32.490...",0.0
2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,1.425185,0.186749,0.417114,0.076406,0.190803,-0.016302,0.075038,0.107870,0.216874,0.095604,0.020977,-0.037011,0.007143,0.002960,0.220526,0.005639,0.010151,0.007453,0.043907,0.001240,0.004347,0.007989,0.017622,0.002636,0.066049,0.032920,0.037618,0.117704,0.041509,0.022645,0.340130,0.007697,0.085784,1.028740,0.449133,0.297935,0.266731,0.258299,0.275012,0.218368,0.198390,0.210177,0.212533,0.204458,0.197634,0.164910,0.007836,0.003079,0.093865,0.005692,0.008212,0.005451,0.042900,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom,"[0.0853077737447, 0.128525418596, 3.1238373468...","[1.6392370551487672, 2.4603095954731096, 31.77...",0.0
3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,1.463686,0.226548,0.404531,0.117699,0.081861,0.053974,0.164865,0.014919,0.117090,0.027778,-0.063173,-0.052606,0.010724,0.003340,0.125459,0.005728,0.014695,0.006322,0.072154,0.001628,0.003493,0.011463,0.032204,0.004738,0.046159,0.036349,0.061960,0.134908,0.032564,0.020036,0.365068,0.005215,0.086336,0.769981,0.425496,0.245312,0.260132,0.224220,0.207597,0.199472,0.207818,0.189912,0.185509,0.187273,0.177629,0.164740,0.008330,0.003528,0.061426,0.005443,0.012382,

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.62176373642359...|  0.0|
|[2.24896123712250...|  0.0|
|[1.63923705514876...|  0.0|
|[1.98016236288167...|  0.0|
|[2.91557666352305...|  0.0|
|[2.44127481235333...|  0.0|
|[2.37111042283127...|  0.0|
|[2.69070524221297...|  0.0|
|[2.26042682391396...|  0.0|
|[2.64022209222507...|  0.0|
|[2.86002034317062...|  0.0|
|[2.30292299045409...|  0.0|
|[1.51190686506125...|  0.0|
|[2.65818792948456...|  0.0|
|[1.94661756778893...|  0.0|
|[2.55302183832814...|  0.0|
|[2.9458137836587,...|  0.0|
|[2.26746739169909...|  0.0|
|[2.13278345629527...|  0.0|
|[2.38849043835619...|  0.0|
+--------------------+-----+
only showing top 20 rows

First row features = [2.621763736423596,1.70103481603904,32.56325622817311,4.349129196001776,9.769208712661392,2.2303314225910627,1.0969734439957743,2.284572126925484,-13.74254910397548,2.089421920313096,0.029811040877083257,0.8793220327583383,0.23157344027726573,0.298682758833

In [7]:
''' split for cross-val '''
trainPerc = 0.7
randSeed = 42

tranBeats, testBeats = beatsML.randomSplit([trainPerc, 1.0 - trainPerc], seed=randSeed)

### Perform the modeling

In [8]:
''' set up the estimators & param grids '''
models = {}

# logistic regression
logreg = LogisticRegression()
params = (ParamGridBuilder().addGrid(logreg.threshold, [0.4, 0.5, 0.6])\
         .addGrid(logreg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
paramNames = ['threshold', 'elasticnetparam']
models['logistic regression'] = [logreg, params, paramNames, None, None]

# random forest
ranfor = RandomForestClassifier(numTrees=20)
params = (ParamGridBuilder().addGrid(ranfor.maxBins, [20, 40, 80, 100])\
              .addGrid(ranfor.maxDepth, [5, 10, 30]).build())
paramNames = ['maxbins', 'maxdepth']
models['random forest'] = [ranfor, params, paramNames, None, None]

In [9]:
''' run the models '''
# number of cv folds
folds = 5
# define the evaulation function
acc = MulticlassClassificationEvaluator(metricName='accuracy')

# iterate over models
for (model, stuff) in models.items():
    print('Cross Validator: %s'%model)
    # execute
    cv = CrossValidator(estimator=stuff[0], estimatorParamMaps=stuff[1], evaluator=acc, numFolds=folds)
    fitModel = cv.fit(tranBeats.select('features', 'label'))
    # get the best
    bestModel = fitModel.bestModel
    # evaluate performance on the test set
    testAcc = acc.evaluate(bestModel.transform(testBeats.select('features', 'label')))
    print('\tBest Model Test Accuracy = %0.3f'%testAcc)    
    # get best parameters
    bestParams = bestModel.extractParamMap()
    for (key, val) in bestParams.items():
        for parm in stuff[2]:
            if parm in key.name.lower():
                print('\t%s = %0.2f'%(key, val))
                break
    # save stuff
    models[model][3] = fitModel
    models[model][4] = testAcc

Cross Validator: logistic regression
	Best Model Test Accuracy = 0.423
	LogisticRegression_06d2f246aa14__elasticNetParam = 0.00
	LogisticRegression_06d2f246aa14__threshold = 0.40
Cross Validator: random forest
	Best Model Test Accuracy = 0.431
	RandomForestClassifier_c437db5d0949__maxBins = 100.00
	RandomForestClassifier_c437db5d0949__maxDepth = 30.00


In [46]:
# show logistic regression summary
bm = models['logistic regression'][3].bestModel
summ = bm.summary
summ.predictions.describe().show()
summ.objectiveHistory

# get models coefficients
coefs = bm.coefficientMatrix.toArray()
ints = bm.interceptVector.toArray()
coefs = pd.concat([pd.DataFrame(index=['Intercept'], data=np.atleast_2d(ints), columns=origLabels),
                   pd.DataFrame(index=inpColumns, data=coefs.T, columns=origLabels)])
display(coefs)

+-------+------------------+------------------+
|summary|             label|        prediction|
+-------+------------------+------------------+
|  count|              1671|              1671|
|   mean|11.128665469778575|11.028126870137642|
| stddev| 6.658178390179078| 6.687000439099515|
|    min|               0.0|               0.0|
|    max|              22.0|              22.0|
+-------+------------------+------------------+



,BigRoom,Breaks,Dance,DeepHouse,DrumAndBass,Dubstep,ElectroHouse,ElectronicaDowntempo,FunkRAndB,FutureHouse,GlitchHop,HardDance,HardcoreHardTechno,HipHop,House,IndieDanceNuDisco,Minimal,ProgressiveHouse,PsyTrance,ReggaeDub,TechHouse,Techno,Trance
Intercept,-0.048447,-0.056893,0.013196,-0.072901,0.035964,-0.006331,-0.020054,0.019095,0.018819,-0.146920,0.100797,-0.002504,0.012057,-0.156298,-0.004645,0.032328,0.029875,0.096520,-0.005594,0.079799,-0.008315,-0.004978,0.095430
1-ZCRm,-0.141024,-0.477734,-0.024363,-0.659684,-0.002382,0.064374,-0.138261,-0.455883,0.287959,-0.022761,0.193358,0.097062,1.039023,0.095852,0.300771,-0.069213,-0.172819,-0.587159,0.184126,-0.125027,0.323808,0.227697,0.062280
2-Energym,1.069079,-0.071711,-0.334947,-1.256720,1.755580,1.016114,1.134163,-1.446987,-1.642539,0.855424,0.138550,1.084068,1.613849,0.727234,0.044468,-2.081860,0.107634,-1.295906,-0.106940,-0.474135,-0.427100,-0.084325,-0.322995
3-EnergyEntropym,0.566687,-0.300965,0.121736,-0.473466,0.013393,-0.119391,0.400959,-0.163203,-0.432386,0.871070,0.064594,0.039546,-0.043347,0.088884,0.135741,-0.403669,-0.023911,-0.207743,0.015111,-0.455079,-0.269533,0.054541,0.520430
4-SpectralCentroidm,-0.053623,0.309036,-0.145210,-0.675569,1.150964,0.385696,0.132284,-0.426058,-0.301059,-0.345229,0.132108,-0.074657,0.135593,0.062839,-0.287804,-0.223020,-0.098373,-0.376007,0.073294,-0.191797,0.263883,0.110068,0.442642
5-SpectralSpreadm,0.566084,0.488293,0.184808,-0.255482,1.572015,-0.275853,0.129838,0.178482,0.118582,-0.948095,-0.101301,-0.040476,-0.249102,-0.845378,-0.759012,0.004047,-0.179332,0.037431,0.016767,-0.120425,-0.305285,-0.062560,0.845952
6-SpectralEntropym,0.080930,-0.143668,-0.281407,-0.750990,-0.380250,0.261135,0.417929,-0.537416,0.078778,0.092829,0.382472,0.123009,0.207526,0.531688,0.121716,-0.135667,-0.200896,-0.421312,0.475827,-0.038856,-0.070973,0.194642,-0.007047
7-SpectralFluxm,-0.400214,-0.256777,-0.368366,-0.188028,-0.213835,-0.741511,-0.301305,-0.730097,-0.609364,-0.189523,-0.314051,-0.266066,1.410775,-0.573605,0.069454,-0.691579,1.251167,0.369974,1.178485,-0.533824,0.745851,0.806647,0.545793
8-SpectralRolloffm,-0.343481,-0.125751,-0.035101,-0.441011,-0.151719,0.513518,0.003443,-0.450055,-0.345043,0.171807,0.084384,-0.050770,0.507087,0.231608,-0.016163,-0.230114,0.188672,-0.194036,0.445195,-0.260938,0.337150,0.138051,0.023268
9-MFCCs1m,1.586168,0.228706,0.518486,-1.164896,0.112025,0.365260,1.210920,-1.108416,-0.587353,1.365690,0.371834,1.026728,-1.407393,0.449521,-0.077827,-0.448446,-0.799099,-0.736297,-0.140974,-0.373963,-0.800479,-0.309101,0.718909


In [ ]:
# view feature importances for random forest
imports = models['random forest'][3].bestModel.featureImportances.toArray()
imports = pd.DataFrame(index=inpColumns, data=imports, columns=['Importance']).sort_values(by='Importance', ascending=False, inplace=False)
display(imports)

In [ ]:
sc.stop()